# Lecture 2: Introduction to Feature Engineering

## Setup

In [42]:
import numpy as np 
import pandas as pd
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [43]:
articles = pd.read_csv('../data/articles.csv')
customers = pd.read_csv('../data/customers.csv')
sample_submisison = pd.read_csv('../data/sample_submission.csv')
transactions = pd.read_csv('../data/transactions_train.csv')

## The H&M Dataset

In [44]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

In [45]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355971 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 73.3+ MB


In [46]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        int64  
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ GB


In [47]:
# X = transactions.merge(customers, how='inner', on='customer_id')
# X = X.merge(articles, how='inner', on='article_id')

### Creating Samples 
If you would rather work with samples instead of the whole dataset (while prototyping your code). You can use the code below:

In [48]:
# Adapted from: https://www.kaggle.com/code/paweljankiewicz/hm-create-dataset-samples
# This extracts three sampled datasets, containing 0.1%, 1% and 5% of all users and their transactions, and the associated articles.
for sample_repr, sample in [("01", 0.001), ("1", 0.01), ("5", 0.05)]:
    break
    print(sample)
    customers_sample = customers.sample(int(customers.shape[0]*sample), replace=False)
    customers_sample_ids = set(customers_sample["customer_id"])
    transactions_sample = transactions[transactions["customer_id"].isin(customers_sample_ids)]
    articles_sample_ids = set(transactions_sample["article_id"])
    articles_sample = articles[articles["article_id"].isin(articles_sample_ids)]
    customers_sample.to_csv(f"../data/customers_sample{sample_repr}.csv.gz", index=False)
    transactions_sample.to_csv(f"../data/transactions_sample{sample_repr}.csv.gz", index=False)
    articles_sample.to_csv(f"../data/articles_sample{sample_repr}.csv.gz", index=False)

In [49]:
articles_sample = pd.read_csv('../data/articles_sample1.csv.gz')
customers_sample = pd.read_csv('../data/customers_sample1.csv.gz')
transactions_sample = pd.read_csv('../data/transactions_sample1.csv.gz')

In [50]:
customers_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13719 entries, 0 to 13718
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_id             13719 non-null  object 
 1   FN                      4781 non-null   float64
 2   Active                  4680 non-null   float64
 3   club_member_status      13658 non-null  object 
 4   fashion_news_frequency  13574 non-null  object 
 5   age                     13559 non-null  float64
 6   postal_code             13719 non-null  object 
dtypes: float64(3), object(4)
memory usage: 750.4+ KB


In [51]:
transactions_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312405 entries, 0 to 312404
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   t_dat             312405 non-null  object 
 1   customer_id       312405 non-null  object 
 2   article_id        312405 non-null  int64  
 3   price             312405 non-null  float64
 4   sales_channel_id  312405 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 11.9+ MB


## A Simplified Task: Binary Classification

The task of predicting which 12 items users are most likely to buy in the next week is difficult to translate to a traditional classification machine learning setting. 
To obtain the 12 items a user is most likely to buy, we need to make predictions for all items (or the ones selected by a baseline) and select the 12 that have the highest predicted scores.

In this assignment, we'll consider a simplified task: Predict whether a user ordered a single item or not, based on the features of the user and the item. 
We provide a baseline logistic regression model below, but haven't done much feature preprocessing or engineering!
Initially, it is always best to focus your efforts on getting your features in the right shape and setting up the right validation scheme and baselines.
Once you are sure that your features add value and your validation scheme is correct, then you typically move on to trying more elaborate models.

### Creating the Dataset

In [52]:
# If you'd rather use a sample. Uncomment the following code:
transactions = transactions_sample
customers = customers_sample
articles = articles_sample

In [53]:
transactions['ordered'] = 1

The problem setting is an example of a "PU learning" problem, i.e. only positives are labeled, everything else is unlabeled (and can be either positive or negative). 
Of course, we cannot train a classifier with just positive samples: The classifier will just learn that everything is positive.
Therefore, we need to manually generate negative samples.

Below, we use a simple random negative sampling strategy.
We want to create a balanced dataset, meaning that we have just as many positives as negatives.
This makes sure that the classifier will not benefit from predicting the positive/negative class more often than the other.
Realistically, the amount of positive samples is of course many times smaller than the amount of unlabeled, possibly negative instances.


If you want to try your hand at a more complex negative sampling strategy, you may want to check out this blog as a starting point: https://medium.com/mlearning-ai/overview-negative-sampling-on-recommendation-systems-230a051c6cd7.



In [54]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,1
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,1
2,2018-09-20,0137b87739a796f65396d8483173f66318039d19a2583f...,577992001,0.011254,2,1
3,2018-09-20,023807eac529e635ef40b48d61685876c7487d129eb238...,524825006,0.023712,2,1
4,2018-09-20,023807eac529e635ef40b48d61685876c7487d129eb238...,524825006,0.023712,2,1


In [55]:
# What's happening here? 
# We're creating negative samples. I.e. we're creating transactions that didn't actually occur.
# First, we need to know which interactions did occur:
positive_pairs = list(map(tuple, transactions[['customer_id', 'article_id']].drop_duplicates().values))

In [56]:
# Then we need to know what every synthetic transaction should contain: a date, a customer_id, an article_id, price, sales_channel_id. We will set ordered = 0, as these transactions didn't really occur.
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,1
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,1
2,2018-09-20,0137b87739a796f65396d8483173f66318039d19a2583f...,577992001,0.011254,2,1
3,2018-09-20,023807eac529e635ef40b48d61685876c7487d129eb238...,524825006,0.023712,2,1
4,2018-09-20,023807eac529e635ef40b48d61685876c7487d129eb238...,524825006,0.023712,2,1


In [57]:
# Extract real values
real_dates = transactions["t_dat"].unique()
real_customers = transactions["customer_id"].unique()
real_articles = transactions["article_id"].unique()
real_channels = transactions["sales_channel_id"].unique()
article_and_price = transactions[["article_id","price"]].drop_duplicates("article_id").set_index("article_id").squeeze()

In [58]:
# How many negatives do we need to sample?
num_neg_pos = transactions.shape[0]
print(num_neg_pos)

312405


In [59]:
# Sampling negatives by selecting random users, articles, dates and sales channel:
# Note: This is quite naive. Some articles may not even have been available at the date we are sampling.
random.seed(42)

# Afterwards, we need to remove potential duplicates, so we'll sample too many.
num_neg_samples = int(num_neg_pos * 1.1)

# Sample each of the independent attributes.
neg_dates = np.random.choice(real_dates, size=num_neg_samples)
neg_articles = np.random.choice(real_articles, size=num_neg_samples)
neg_customers = np.random.choice(real_customers, size=num_neg_samples)
neg_channels = np.random.choice(real_channels, size=num_neg_samples)
ordered = np.array([0] * num_neg_samples)
# Assign to every article a real price.
neg_prices = article_and_price[neg_articles].values

In [60]:
neg_transactions = pd.DataFrame([neg_dates, neg_customers, neg_articles, neg_prices, neg_channels, ordered], index=transactions.columns).T

In [61]:
# Result:
neg_transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-10-03,9aac1f83aea5da3cf70762ae96815147688002519a4a0f...,770315020,0.016932,1,0
1,2019-02-26,8c331ca8e900be944ec1f71ece181bce3f4667963c7e82...,626887003,0.022017,1,0
2,2020-08-02,93a220587363727c45ebe210604908205544412bcb6e73...,878629003,0.016932,2,0
3,2019-01-08,62ca22c014275d8b5fe7b5aee291b427303ad7747aa117...,677885019,0.003373,2,0
4,2019-08-07,415efde87c0f423ffa59eae284a98dfca4fbf6875dfb9e...,824764012,0.025407,2,0


In [62]:
neg_transactions.shape

(343645, 6)

In [63]:
# Remove random negative samples that actually coincide with positives
df = neg_transactions[
    ~neg_transactions.set_index(["customer_id", "article_id"]).index.isin(positive_pairs)
]

# Remove any excess
chosen_neg_transactions = df.sample(num_neg_pos)

In [64]:
# Concat the negative samples to the positive samples:
transactions = pd.concat([transactions, chosen_neg_transactions])
transactions = transactions.merge(customers, how="inner", on='customer_id')
transactions = transactions.merge(articles, how="inner", on='article_id')

In [65]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 624810 entries, 0 to 624809
Data columns (total 36 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   t_dat                         624810 non-null  object 
 1   customer_id                   624810 non-null  object 
 2   article_id                    624810 non-null  object 
 3   price                         624810 non-null  object 
 4   sales_channel_id              624810 non-null  object 
 5   ordered                       624810 non-null  object 
 6   FN                            241239 non-null  float64
 7   Active                        237581 non-null  float64
 8   club_member_status            622759 non-null  object 
 9   fashion_news_frequency        620320 non-null  object 
 10  age                           619882 non-null  float64
 11  postal_code                   624810 non-null  object 
 12  product_code                  624810 non-nul

### Basic Preprocessing
Some very basic preprocessing.

In [66]:
# I'm dropping a lot of columns, use them in your engineering tasks!
transactions_processed = transactions[['customer_id', 'age', 'article_id', 'sales_channel_id', 'price', 'ordered', 'section_no', 'product_type_no', 'garment_group_no']].copy()
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered,section_no,product_type_no,garment_group_no
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,24.0,663713001,2,0.050831,1,61,283,1017
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,24.0,663713001,2,0.050831,1,61,283,1017
2,4ef5967ff17bf474bffebe5b16bd54878e1d4105f7b4ed...,37.0,663713001,2,0.050831,1,61,283,1017
3,b1fc3f8a8b0b3f8ae55539720164c4a41ef82b03ef1e06...,23.0,663713001,1,0.050831,0,61,283,1017
4,34c2e199bd667e43ad0c1d0910afe85fe8e315e13c5503...,20.0,663713001,1,0.050831,1,61,283,1017


In [67]:
# Does it make sense to label encode?
# Label encoding the customer and article IDs:
customer_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()
section_no_encoder = preprocessing.LabelEncoder()
product_type_no_encoder = preprocessing.LabelEncoder()
garment_group_no_encoder = preprocessing.LabelEncoder()

In [68]:
# Important: comment / uncomment the ones that should be label-encoded
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])
# transactions_processed['section_no'] = section_no_encoder.fit_transform(transactions_processed['section_no'])
# transactions_processed['product_type_no'] = product_type_no_encoder.fit_transform(transactions_processed['product_type_no'])
# transactions_processed['garment_group_no'] = garment_group_no_encoder.fit_transform(transactions_processed['garment_group_no'])

# Additionally: comment / uncomment the columns that are not going to be used
transactions_processed.drop(['section_no'], axis=1, inplace=True)
transactions_processed.drop(['product_type_no'], axis=1, inplace=True)
# transactions_processed.drop(['garment_group_no'], axis=1, inplace=True)

In [69]:
# If you want to go back to the original encoding:
customer_encoder.inverse_transform([2])

array(['0005476832b9770a320d94de5518f78587ed4aa848b2a7c7407f12f251fcd5aa'],
      dtype=object)

In [70]:
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered,garment_group_no
0,0,24.0,16192,2,0.050831,1,1017
1,0,24.0,16192,2,0.050831,1,1017
2,4175,37.0,16192,2,0.050831,1,1017
3,9409,23.0,16192,1,0.050831,0,1017
4,2784,20.0,16192,1,0.050831,1,1017


In [71]:
# Can you come up with a NaN strategy that makes sense for each column in the dataset?
# Imputing all NaN values with zeros:
transactions_processed = transactions_processed.fillna(0)
transactions_processed.isnull().values.any()

False

In [72]:
# Does it make sense to one-hot encode?
# One-hot-encoding sales_channel_id:
# Important: comment / uncomment the ones that should be one-hot encoded
transactions_processed = pd.get_dummies(transactions_processed, columns=['sales_channel_id'])
# transactions_processed = pd.get_dummies(transactions_processed, columns=['section_no'])
# transactions_processed = pd.get_dummies(transactions_processed, columns=['product_type_no'])
transactions_processed = pd.get_dummies(transactions_processed, columns=['garment_group_no'])

In [73]:
transactions_processed.head()

,customer_id,age,article_id,price,ordered,sales_channel_id_1,sales_channel_id_2,garment_group_no_1001,garment_group_no_1002,garment_group_no_1003,...,garment_group_no_1013,garment_group_no_1014,garment_group_no_1016,garment_group_no_1017,garment_group_no_1018,garment_group_no_1019,garment_group_no_1020,garment_group_no_1021,garment_group_no_1023,garment_group_no_1025
0,0,24.0,16192,0.050831,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,24.0,16192,0.050831,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,4175,37.0,16192,0.050831,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,9409,23.0,16192,0.050831,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,2784,20.0,16192,0.050831,1,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [74]:
# Creating a Train / Test Split:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)

In [75]:
X_train.head()

,customer_id,age,article_id,price,sales_channel_id_1,sales_channel_id_2,garment_group_no_1001,garment_group_no_1002,garment_group_no_1003,garment_group_no_1005,...,garment_group_no_1013,garment_group_no_1014,garment_group_no_1016,garment_group_no_1017,garment_group_no_1018,garment_group_no_1019,garment_group_no_1020,garment_group_no_1021,garment_group_no_1023,garment_group_no_1025
529657,12352,23.0,20523,0.025407,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
530797,2215,23.0,28470,0.016932,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
274095,5024,64.0,12250,0.024525,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
96800,8411,49.0,6617,0.033881,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
81298,13149,43.0,33678,0.030492,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
y_train.head()

529657    1
530797    1
274095    0
96800     1
81298     1
Name: ordered, dtype: int64

## Baseline Model

In [77]:
# Will take a few minutes to run, if you're using the whole dataset:
baseline = LogisticRegression(random_state=42)
baseline = baseline.fit(X_train, y_train)

In [78]:
baseline.predict_proba(X_test)

array([[0.44265649, 0.55734351],
       [0.42770108, 0.57229892],
       [0.38102396, 0.61897604],
       ...,
       [0.40224831, 0.59775169],
       [0.42779364, 0.57220636],
       [0.39887692, 0.60112308]])

In [79]:
y_test

94036     1
584831    0
602078    0
275287    1
602429    0
         ..
276099    0
168323    1
534506    1
577216    0
43626     1
Name: ordered, Length: 62481, dtype: int64

In [80]:
# Mean Accuracy:
baseline.score(X_test, y_test)
# As you can seen, the accuracy is ~0.51. In other words, the classifier predicts correctly 51% of the time whether a customer did or din't buy an item.
# Can you improve this baseline logistic regression model by doing better preprocessing and generating new features?
# Also think about my steps! Did it make sense to include the article and customer ids? (And things like that)

0.5982138570125318

In [81]:
# Classification Metrics:
predictions = baseline.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.62      0.50      0.55     31139
           1       0.58      0.70      0.64     31342

    accuracy                           0.60     62481
   macro avg       0.60      0.60      0.59     62481
weighted avg       0.60      0.60      0.59     62481



In [82]:
predictions

array([1, 1, 1, ..., 1, 1, 1])

## Assignment: Feature engineering
**TODO:** 
- In groups (of 2-3 students), think about (a few) features that can be engineered (preprocess and generate new features). Divide the work!
- Do these engineered features improve the baseline model?
- Add your thoughts & results to a slide deck for discussion next week (again, 1 slide per person).
